In [2]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [3]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [5]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:4000,
    GenParams.MIN_NEW_TOKENS:1,
    # "temperature":0.1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```","[/INST]","<|user|>","<|endoftext|>","<|assistant|>"],
}

model = ModelInference(model_id="ibm/granite-3-8b-instruct", credentials=creds, params=params, project_id=project_id)

In [20]:
model_id = GRANITE_3_8B_INSTRUCT
language = "golang"
documentframework = "godoc"
testframework = "testing"

In [8]:
prompt = f"""<|user|>
Please generate a sample {language} code:
-give more detail and complex logic.
-generate the code only.
-dont generate explanation.
<|assistant|>
```{language}
"""

sample_code = model.generate_text(prompt).replace("```","")
print(sample_code)

package main

import (
	"fmt"
	"sync"
)

type Worker struct {
	ID     int
	tasks  chan int
	done   chan struct{}
	wg     sync.WaitGroup
}

func NewWorker(id int, tasks chan int, done chan struct{}) *Worker {
	return &Worker{
		ID:     id,
		tasks:  tasks,
		done:   done,
		wg:     sync.WaitGroup{},
	}
}

func (w *Worker) Start() {
	for task := range w.tasks {
		w.wg.Add(1)
		go func(task int) {
			defer w.wg.Done()
			fmt.Printf("Worker %d processing task %d\n", w.ID, task)
			// Simulate task processing time
			time.Sleep(1 * time.Second)
			fmt.Printf("Worker %d finished task %d\n", w.ID, task)
		}(task)
	}
}

func (w *Worker) Stop() {
	close(w.done)
	w.wg.Wait()
}

func main() {
	tasks := make(chan int, 10)
	done := make(chan struct{})

	for i := 1; i <= 5; i++ {
		worker := NewWorker(i, tasks, done)
		go worker.Start()
	}

	// Send tasks to workers
	for i := 1; i <= 10; i++ {
		tasks <- i
	}

	// Close tasks channel and wait for all workers to finish
	close(tasks)
	for i := 1; i <=

In [9]:
sample_code = """package main

import (
	"fmt"
	"sync"
)

type Worker struct {
	ID     int
	tasks  chan int
	done   chan struct{}
	wg     sync.WaitGroup
}

func NewWorker(id int, tasks chan int, done chan struct{}) *Worker {
	return &Worker{
		ID:     id,
		tasks:  tasks,
		done:   done,
		wg:     sync.WaitGroup{},
	}
}

func (w *Worker) Start() {
	for task := range w.tasks {
		w.wg.Add(1)
		go func(task int) {
			defer w.wg.Done()
			fmt.Printf("Worker %d processing task %d\n", w.ID, task)
			// Simulate task processing time
			time.Sleep(1 * time.Second)
			fmt.Printf("Worker %d finished task %d\n", w.ID, task)
		}(task)
	}
}

func (w *Worker) Stop() {
	close(w.done)
	w.wg.Wait()
}

func main() {
	tasks := make(chan int, 10)
	done := make(chan struct{})

	for i := 1; i <= 5; i++ {
		worker := NewWorker(i, tasks, done)
		go worker.Start()
	}

	// Send tasks to workers
	for i := 1; i <= 10; i++ {
		tasks <- i
	}

	// Close tasks channel and wait for all workers to finish
	close(tasks)
	for i := 1; i <= 5; i++ {
		<-done
	}

	fmt.Println("All tasks completed")
}
"""

bad_code = """package main

import (
	"fmt"
	"sync"
)

type Worker struct {
	ID     int
	tasks  chan int
	done   chan struct{}
	wg     sync.WaitGroup
}

func NewWorker(id int, tasks chan int, done chan struct{}) *Worker {
	return &Worker{
		ID:     id,
		tasks:  tasks,
		wg:     sync.WaitGroup{},
	}
}

func (w *Worker) Start() {
	for task := range w.tasks {
		w.wg.Add(1)
		go func(task int) {
			defer w.wg.Done()ssing time
			time.Sleep(1 * time.Second)
			fmt.Printf("Worker %d finished task %d\n", w.ID, task)
		}(task)
	}
}

func (w *Worker) Stop() {
	close(w.done)
	w.wg.Wait()
}

func main() {
	tasks := make(chan int, 10)
	done := make(chan struct{})

	for i := 1; i <= 5; i++ {
		worker := New
        er.Start()
	}

	// Send tasks to workers
	for i := 1; i

	// Close tasks channel and wait for all workers to finish
	close(tasks)
	for i := 1; i <= 5; i++ {
		<-done
	}

	fmt.Println("All tasks completed")
}
"""

gnerate explanation

In [10]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = model.generate_text(prompt)
print(answer)


This Go code is a simple example of a worker pool pattern, where multiple workers process tasks concurrently. Here's a detailed explanation:

1. The `Worker` struct represents a worker with an ID, a channel for receiving tasks, a channel for signaling completion, and a `sync.WaitGroup` for waiting on all tasks to finish.

2. The `NewWorker` function creates a new worker with the given ID, tasks channel, and done channel.

3. The `Start` method of the `Worker` struct processes tasks from the tasks channel. For each task, it adds a goroutine to the `sync.WaitGroup`, processes the task (simulating processing time with `time.Sleep`), and then calls `wg.Done()` to signal that the task is finished.

4. The `Stop` method of the `Worker` struct closes the done channel and waits for all tasks to finish using `wg.Wait()`.

5. In the `main` function, a tasks channel and a done channel are created. Five workers are created and started using the `NewWorker` and `Start` functions. Ten tasks are sen

generate godoc

In [36]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
{language}: `{sample_code}`
-generate inline comment with code.
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


// Package main provides a simple example of a worker pool in Go.
// The Worker struct represents a worker that processes tasks from a channel.
// The NewWorker function creates a new worker with the given ID, tasks channel, and done channel.
// The Start method starts processing tasks from the tasks channel.
// The Stop method closes the done channel and waits for all workers to finish.
// The main function creates a worker pool with 5 workers, sends 10 tasks to the workers, and waits for all workers to finish.
package main

import (
	"fmt"
	"sync"
	"time"
)

// Worker represents a worker that processes tasks from a channel.
type Worker struct {
	ID     int
	tasks  chan int
	done   chan struct{}
	wg     sync.WaitGroup
}

// NewWorker creates a new worker with the given ID, tasks channel, and done channel.
func NewWorker(id int, tasks chan int, done chan struct{}) *Worker {
	return &Worker{
		ID:     id,
		tasks:  tasks,
		done:   done,
		wg:     sync.WaitGroup{},
	}
}

// Start start

score, review and improve

In [37]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-make it more object oriented.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
-dont generate code.
<|assistant|>
score and review:"""

suggestion = model.generate_text(prompt)
print(suggestion)



The provided Go code is a simple implementation of a worker pool using goroutines and channels. It creates a specified number of workers, each processing tasks from a shared channel. The code is generally well-structured and easy to understand, but there are some areas for improvement in terms of object-oriented design, error handling, and performance.

Here's a detailed review and suggestions for improvement:

1. Object-oriented design:
   - The `Worker` struct is a good start, but it could be improved by adding methods for task processing and error handling.
   - Consider creating a `Task` struct to encapsulate task-related data and behavior.
   - Add a `ProcessTask` method to the `Worker` struct to handle task processing logic.

2. Error handling:
   - The code currently lacks error handling. For example, there's no error checking when creating goroutines or when processing tasks.
   - Add error handling to make the code more robust and easier to debug.

3. Performance:
   - The c

In [38]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
-dont generate explanation.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


package main

import (
	"fmt"
	"sync"
	"time"
)

type Task struct {
	ID     int
	data   interface{}
}

type Worker struct {
	ID     int
	tasks  chan Task
	done   chan struct{}
	pool   sync.Pool
	wg     sync.WaitGroup
}

func NewWorker(id int, tasks chan Task, done chan struct{}) *Worker {
	return &Worker{
		ID:     id,
		tasks:  tasks,
		done:   done,
		pool:   sync.Pool{New: func() interface{} { return NewWorker(id, tasks, done) }},
		wg:     sync.WaitGroup{},
	}
}

func (w *Worker) Start() {
	for task := range w.tasks {
		w.wg.Add(1)
		go func(task Task) {
			defer w.wg.Done()
			err := w.ProcessTask(task)
			if err != nil {
				fmt.Printf("Worker %d failed to process task %d: %v\n", w.ID, task.ID, err)
			}
		}(task)
	}
}

func (w *Worker) ProcessTask(task Task) error {
	fmt.Printf("Worker %d processing task %d with data: %v\n", w.ID, task.ID, task.data)
	// Simulate task processing time
	time.Sleep(1 * time.Second)
	fmt.Printf("Worker %d finished task %d\n", w.ID, task.ID)
	return

fix bad code

In [39]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


package main

import (
	"fmt"
	"sync"
	"time"
)

type Worker struct {
	ID     int
	tasks  chan int
	done   chan struct{}
	wg     sync.WaitGroup
}

func NewWorker(id int, tasks chan int, done chan struct{}) *Worker {
	return &Worker{
		ID:     id,
		tasks:  tasks,
		wg:     sync.WaitGroup{},
		done:   done, // add done channel to Worker struct
	}
}

func (w *Worker) Start() {
	for task := range w.tasks {
		w.wg.Add(1) // increment wait group counter
		go func(task int) {
			defer w.wg.Done() // decrement wait group counter when done
			time.Sleep(1 * time.Second) // sleep for 1 second
			fmt.Printf("Worker %d finished task %d\n", w.ID, task)
		}(task)
	}
}

func (w *Worker) Stop() {
	close(w.done) // close done channel to signal workers to stop
	w.wg.Wait()   // wait for all workers to finish
}

func main() {
	tasks := make(chan int, 10) // create a buffered channel with capacity 10
	done := make(chan struct{}) // create a done channel

	for i := 1; i <= 5; i++ {
		worker := NewWorker(

generate diagram

In [40]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


graph TD
A[main] --> B[tasks := make(chan int, 10)]
A --> C[done := make(chan struct{})]
A --> D[for i := 1; i <= 5; i++ { worker := NewWorker(i, tasks, done) go worker.Start() }]
A --> E[for i := 1; i <= 10; i++ { tasks <- i }]
A --> F[close(tasks)]
A --> G[for i := 1; i <= 5; i++ { <-done }]
A --> H[fmt.Println("All tasks completed")]



conversion

In [44]:
target_language = "rust"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


use std::sync::mpsc;
use std::sync::Arc;
use std::sync::Mutex;
use std::thread;
use std::time::Duration;

struct Worker {
    id: u32,
    tasks: mpsc::Receiver<u32>,
    done: Arc<Mutex<bool>>,
}

impl Worker {
    fn new(id: u32, tasks: mpsc::Receiver<u32>, done: Arc<Mutex<bool>>) -> Self {
        Self { id, tasks, done }
    }

    fn start(&self) {
        for task in self.tasks.iter() {
            let done = self.done.clone();
            thread::spawn(move || {
                println!("Worker {} processing task {}", self.id, task);
                std::thread::sleep(Duration::from_secs(1));
                println!("Worker {} finished task {}", self.id, task);
                *done.lock() = true;
            });
        }
    }

    fn stop(&self) {
        drop(self.tasks);
    }
}

fn main() {
    let (tasks_tx, tasks_rx) = mpsc::channel();
    let done = Arc::new(Mutex::new(false));

    for i in 1..=5 {
        let tasks_tx = tasks_tx.clone();
        let done = done.clon

generate api definition

In [42]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
-dont generate explanation.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = model.generate_text(prompt).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: Worker API
  version: 1.0.0
servers:
  - url: http://localhost:8080/api
paths:
  /start:
    post:
      summary: Start a new worker
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                id:
                  type: integer
                  minimum: 1
                tasks:
                  type: array
                  items:
                    type: integer
                  minItems: 1
                  maxItems: 10
                done:
                  type: array
                  items:
                    type: string
                  minItems: 1
                  maxItems: 5
      responses:
        '200':
          description: Worker started successfully
/stop:
  post:
    summary: Stop a worker
    requestBody:
      required: true
      content:
        application/json:
          schema:
            type: object
      

generate test case

In [43]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
-generate the test only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


package main

import (
	"fmt"
	"sync"
	"testing"
	"time"
)

func TestWorker_Start(t *testing.T) {
	tasks := make(chan int, 10)
	done := make(chan struct{})

	for i := 1; i <= 5; i++ {
		worker := NewWorker(i, tasks, done)
		go worker.Start()
	}

	// Send tasks to workers
	for i := 1; i <= 10; i++ {
		tasks <- i
	}

	// Close tasks channel and wait for all workers to finish
	close(tasks)
	for i := 1; i <= 5; i++ {
		<-done
	}

	// Check if all tasks are processed
	for i := 1; i <= 10; i++ {
		select {
		case <-done:
			continue
		default:
			t.Errorf("Task %d not processed", i)
		}
	}
}

func TestWorker_Stop(t *testing.T) {
	tasks := make(chan int, 10)
	done := make(chan struct{})

	worker := NewWorker(1, tasks, done)
	go worker.Start()

	// Stop worker before any tasks are processed
	worker.Stop()

	// Check if done channel is closed
	select {
	case <-done:
	default:
		t.Error("Done channel not closed")
	}
}

func TestWorker_Start_EmptyTasks(t *testing.T) {
	tasks := make(chan int, 10